[View in Colaboratory](https://colab.research.google.com/github/InnovArul/nettutorial/blob/master/deep-ai-explanation/deep_explanation.ipynb)

In [1]:
# install torch
!pip3 install torchvision

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

# http://pytorch.org/
from os import path
import os
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(os.listdir('drive/'))

# check torch
import torch
print(torch.cuda.is_available())

# 4/AAArEUBolsikLH79NWhuPvgc_42PEFTdk_LSx5LaSgacoC4fFzh9q8Q
# 4/AAD53xotTis4sv1mTFxudK8k9WlBQ0Ng81S2KMg-gjFu7RgQ_yP6ewo


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
['.Trash', 'mnist', 'Colab Notebooks', 'MoneyManager', 'SmsContactsBackup', 'visa rejection.pdf', 'Multiple View Geometry in Computer Vision (Second Edition).pdf', 'mvf-tutorial.pdf', 'Deep learning workshop (Responses).ods', 'jstock-fe78440e-e0fe-4efb-881d-264a01be483c-checksum=2251512174-date=1513596102632-v

In [1]:
%matplotlib inline

# import statements
import torch, os, sys
import torch.nn as nn, numpy as np
import torch.nn.functional as F
import torchvision, torchvision.models as models
import matplotlib.pyplot as plt, numpy as np
from skimage import io, transform, morphology
from tqdm import tqdm

# ignore warnings
import warnings
warnings.filterwarnings('ignore')



ModuleNotFoundError: ignored

In [2]:
# hyper parameters
image_path = ''
kerne_size = 3
n_blocks = 3
n_layers = 3
epochs = 1000
lr = 0.01
l1_coeff = 1.0
center_of_mass = True

In [3]:
# helper functions
def use_cuda():
    return torch.cuda.is_available()

def image_torch_to_np(img):
    img = img.data.cpu().numpy()[0]
    img = img.transpose((1,2,0))
    return img

# load the model
def load_model(model_type='vgg16'):
    model = models[model_type](pretrained=True)
    model.eval()
    
    if use_cuda():
        model = model.cuda()
    
    for p in model.features.parameters():
        p.requires_grad = False
    for p in model.classifier.parameters():
        p.requires_grad = False
       
    return model

model = load_model()
print(model)

NameError: ignored

In [0]:
def load_image(img_path):
    img = io.imread(img_path)
    
    if show_image:
        fig = plt.figure('image')
        fig.imshow(img)
    
    return img

img_np = load_image(image_path)

In [0]:
def image_np_to_torch(img):
    resized_img = np.float32(transform.resize(img, (224, 224)))
    means=[0.485, 0.456, 0.406]
    stds=[0.229, 0.224, 0.225]
    
    for ch in range(3):
        resized_img[:, :, ch] *= (means[ch] / stds[ch])
    
    resized_img = torch.from_numpy(resized_img.transpose((2,0,1))).unsqueeze(0)
    if use_cuda():
        resized_img = resized_img.cuda()
    
    return Variable(resized_img, requires_grad=True)

img_torch = image_np_to_torch(img_np)
print type(img_torch)
print img_torch.shape
print((img_torch.min(), img_torch.max()))
    
    
    

In [4]:
# visualize scaling changes
img_np_resized = img_torch_to_np(img_torch)
img_avg = np.mean(img_np_resized, axis=2)

plt.pcolormesh(img_avg)
plt.colorbar()
ax = plt.gca()
ax.set_aspect(1)
__ = plt.title("Vizualization of image transformed to torch tensor")

NameError: ignored

In [0]:
def assess_input(input_img):
    with open("data/imagenet1000_clsid_to_human.pkl", "r") as fp:
        vgg_class = pickle.load(fp)
       
    outputs = F.softmax(model(input_img))
    outputs_np = outputs.data.cpu().numpy()
    sorted_args = np.argsort(outputs_np[0, :])[::-1]
    
    print('top 5 classes')
    print('(index) description : score')
    
    for i in sorted_args:
        print('({}) {} : {}'.format(i, vgg_class[i], outputs_np[0,i]))
    
    if outputs_np[0, sorted_args[0]] < 0.5:
        print("*** Warning ***")
        print("top category score under 0.5, extracted explanation may not be accurate on not well defined class")


vgg_input_assessment(img_torch, model)
    

In [0]:
class PerturbationGenerator(nn.Module):
    def __init__(self, kernel_size=3, n_blocks=3, n_layers=3):
        self(PerturbationGenerator, self).__init__()
        
        # if kernel_size is even
        if kernel_size % 2 == 1:
            kernel_size = kernel_size + 1
        
        self.conv = nn.Conv2d(3, 3, kernel_size=kernel_size, padding=kernel_size//2)
        self.n_blocks = n_blocks
        self.n_layers = n_layers
        
    def forward(img):
        # get scaling information
        x_min = torch.min(img)
        Dx_max = torch.max(img - x_min)
        
        # perform manipulation
        for _ in range(self.n_blocks * self.n_layers):
            x = self.conv(x)
        
        # rescale the manipulated output
        x = (x - torch.min(x)) / torch.max(x) # x : 0 to 1
        x = (x * Dx_max) + x_min # x : original xmin to xmax
        
        return x

# create an instance
img_perturber = PerturbationGenerator(kernel_size, n_blocks, n_layers)

In [0]:
def get_optimum_perturbation(epochs, model, perturber, img, lr=0.1, l1_coeff=0.01):
    optimizer = optim.Adam(perturber.parameters(), lr=lr)
    
    # get the max output
    output = F.softmax(model(img))
    max_category = np.argmax(output.cpu().data.numpy())
    print('category of highest probability', max_category, vgg_class[max_category])
    losses = []
    
    # optimize the perturber for getting adversarial image
    print('optimizing the perturber')
    for _ in tqdm(range(epochs)):
        perturbed_img = perturber(img)
        output = F.softmax(model(perturbed_img))
        img_diff = perturbed_img - img
        loss = torch.mean(torch.abs(img_diff)) + output[0, max_category]
        losses.append(loss.data[0])
        
        # optimize 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
   
    # plot the loss
    plt.figure("loss")
    plt.plot(losses)
    plt.xlabel("epoch")
    plt.ylabel("loss")
    
    print "original score: {}".format(F.softmax()(vgg_model(img))[0, max_category])
    print "perturbed score: {}".format(F.softmax()(vgg_model(perturbed_img))[0, max_category])
    return perturbed_img

perturbed_img = get_optimum_perturbation(
    epochs, model, img_perturber, img_torch,
    lr=lr, l1_coeff=l1_coeff,
)

In [0]:
original_img_np = image_torch_to_np(img_torch)
perturbed_img_np = image_torch_to_np(perturbed_img)

mean_original_np = np.mean(original_img_np, axis=2)
mean_perturbed_np = np.mean(perturbed_img_np, axis=2)

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))
fig.canvas.set_window_title("images")

im1 = ax1.pcolormesh(mean_original_np)
fig.colorbar(im1, ax=ax1, fraction=0.046)
ax1.set_aspect(1)
ax1.set_title("mean original image")

im2 = ax2.pcolormesh(mean_perturbed_np)
fig.colorbar(im2, ax=ax2, fraction=0.046)
ax2.set_aspect(1)
ax2.set_title("mean perturbed image")

fig.tight_layout()


In [0]:
def post_processing(original_img, perturbed_img):
    original_img_np = image_torch_to_np(original_img, axis=2)
    perturbed_img_np = image_torch_to_np(perturbed_img, axis=2)

    mean_original_np = np.mean(original_img_np, axis=2)
    mean_perturbed_np = np.mean(perturbed_img_np, axis=2)    
    diff = (mean_original_np - mean_perturbed_np)**6
    
    # remove perturbations in the edges of image
    h, w = np.shape(diff)
    diff[:int(0.1 * h), :] = 0
    diff[int(0.9 * h):, :] = 0
    diff[:, :int(0.1 * w)] = 0
    diff[:, int(0.9 * w):] = 0
    
    # dilate the important points left for visibility:
    square = np.ones((20, 20))
    diff = morphology.dilation(diff, square)
    return diff, original_img_np, perturbed_img_np, mean_original_np, mean_perturbed_np
    
    
diff, original_img_np, perturbed_img_np, mean_original_np, mean_perturbed_np = post_processing(img_torch, perturbed_img)

In [0]:
def plot_results(diff, original_img_np, perturbed_img_np, mean_original_np, mean_perturbed_np, indicate_center_of_mass=False):
    center_mass = center_of_mass(diff)
    
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(15, 5))
    
    im1 = ax1.pcolormesh(mean_original_np)
    fig.colorbar(im1, ax=ax1, fraction=0.046)
    ax1.set_aspect(1)
    ax1.set_title('original image')
    
    im2 = ax2.pcolormesh(mean_perturbed_np)
    fig.colorbar(im2, ax=ax2, fraction=0.046)
    ax2.set_aspect(1)
    ax2.set_title('perturbed image')
    
    im3 = ax1.pcolormesh(diff)
    fig.colorbar(im3, ax=ax3, fraction=0.046)
    ax3.set_aspect(1)
    ax3.set_title('differences')    
    
    if indicate_center_of_mass:
        ax3.annotate("X: center of mass", center_mass) 
        
    fig.tight_layout()
    plt.show()

plot_results(diff, original_img_np, perturbed_img_np, mean_original_np, mean_perturbed_np, indicate_center_of_mass=False)

In [1]:
def deep_explanation(image_path="drive/image_samples/cat2.jpg",
                    kernel_size=3,
                    nblocks=2,
                    nlayers=3,
                    epochs=1000,
                    lr=0.01,
                    l1_coeff=1.0,
                    indicate_center_of_mass=True,):
    model = load_model()
    print(model)

    img_np = load_image(image_path)
    img_torch = image_np_to_torch(img_np)
    vgg_input_assessment(img_torch, model)
    img_perturber = PerturbationGenerator(kernel_size, n_blocks, n_layers)
    perturbed_img = get_optimum_perturbation(
        epochs, model, img_perturber, img_torch,
        lr=lr, l1_coeff=l1_coeff,
    )

    diff, original_img_np, perturbed_img_np, mean_original_np, mean_perturbed_np = post_processing(img_torch, perturbed_img)
    plot_results(diff, original_img_np, perturbed_img_np, mean_original_np, mean_perturbed_np, indicate_center_of_mass=False)
    

fatal: Not a git repository (or any of the parent directories): .git
